In [ ]:
-- Merges the incident_id from the MCI onto the ecr data store if a matching ID has been found AND the ecr was generated within 90 days of the speciment collection date
MERGE INTO delta.`/delta-tables/ecr_datastore` AS ecr
USING delta.`/delta-tables/mci` AS mci
ON ecr.iris_id = mci.person_id AND DATEDIFF(day,ecr.ecr_date,mci.specimen_collection_date) <= 90
WHEN MATCHED 
    THEN UPDATE SET incident_id = mci.incident_id;